In [ ]:
import json
import numpy as np
from sklearn.linear_model import LinearRegression
import os
import sys
import pandas as pd
import math
from matplotlib import cm
from matplotlib import pyplot as plt
from sklearn.model_selection import LeaveOneOut
# PATH TO PROJECT ROOT
sys.path.append("/path/to/VocalDetection")
from definitions import DATA_DIR

In [ ]:
def avg(cls_results, ground_truth):
    acc = []
    ground_truth = np.array(ground_truth)
    total = ground_truth.shape[0]
    for v in np.array(cls_results):
        acc.append(np.sum(ground_truth[:,0] == v[:,0])/total)

    return np.sum(acc)/len(acc) 

def voting(cls_results, ground_truth, title="Default"):
    """
    To calculate voting accuracy.

        cls_results == np.where(cls.predict(test_ds) >= 0.5, 1, 0)
        ground_truth == [ y.numpy() for _, y in dataset.load(test_ds_path)]
        title(str)
    """
    ground_truth = np.array(ground_truth)
    voting_0_count = np.zeros(ground_truth.shape[0])
    voting_1_count = np.zeros(ground_truth.shape[0])
    cls_results = np.array(cls_results)
    for v in cls_results:
        for i, value in enumerate(v):
            if value[0] == 1:
                voting_1_count[i] += 1
            else:
                voting_0_count[i] += 1

    voting_result_rates = [0 for i in range(cls_results.shape[0]//2 + 1)] # e.g. 7 = 0 1 2 3   21 = 0 1 2 ... 10
    voting_result = np.zeros(ground_truth.shape[0])

    for i in range(ground_truth.shape[0]):
        x, y = voting_0_count[i], voting_1_count[i]
        if x < y:
            voting_result[i] = 1
        voting_result_rates[int(min(x, y))] += 1
    return np.sum(voting_result == ground_truth[:, 0]) / ground_truth.shape[0], voting_result_rates

def classfication_voting(filename, train_list, test_list, tops=4, nums=None, axs=None, df=None, test_df=None, title='Default', debug=False):
    _cls_results = json.load(open(filename))
    _keys = list(_cls_results['ground_truth'].keys())

    total_classfication = np.asarray(_cls_results[_keys[0]]).shape[0]

    if isinstance(nums, int):
        if nums > total_classfication:
            nums = total_classfication + 1
        elif nums <= 0:
            nums = total_classfication + 1
    else:
        nums = total_classfication + 1

    _train_keys = [ key for key in _keys if key in train_list]
    _test_keys = [ key for key in _keys if key in test_list]
    if debug:
        print('train_keys: ' + ', '.join(_train_keys).replace('SCNN-', '').replace('.h5', ''))
        print('test_keys: ' + ', '.join(_test_keys).replace('SCNN-', '').replace('.h5', ''))

    _ground_truth = [ np.asarray(x) for x in _cls_results['ground_truth'].values()]
    _cls_predict = [ np.asarray(list(x)[:nums]) for x in _cls_results.values()][:-1] # last one is ground_truth
    x = []   # Average accuracies
    y = []   # Voting accuracies
    test_x = []
    test_y = []

    top_y = [[] for i in range(tops)] # Top n high Classification confidence
    test_top_y = [[] for i in range(tops)]

    for i, key in enumerate(_keys):
        if key not in _train_keys and key not in _test_keys:
            continue

        total = _ground_truth[i].shape[0]
        acc, acc_rates = voting(_cls_predict[i], _ground_truth[i])
        print(_cls_predict[i].shape)
        exit()
        avg_acc = avg(_cls_predict[i], _ground_truth[i])
        if key in _test_keys:
            test_x.append(avg_acc)
            test_y.append(acc)
            for j in range(tops):
                test_top_y[j].append(np.sum(acc_rates[:j+1])/total)
        elif key in _train_keys:
            x.append(avg_acc)
            y.append(acc)
            for j in range(tops):
                top_y[j].append(np.sum(acc_rates[:j+1])/total)

    x = np.array(x).reshape(-1, 1)
    y = np.array(y).reshape(-1, 1)
    test_x = np.array(test_x).reshape(-1, 1)
    test_y = np.array(test_y).reshape(-1, 1)
    test_top_y = np.array(test_top_y).reshape(tops, -1, 1)
    top_y = np.array(top_y).reshape(tops, -1, 1)
    

    if axs is None:
        fig, axs = plt.subplots(nrows=math.ceil(tops/2), ncols=2, figsize=(18, 4*math.ceil(tops/2)))
        fig.suptitle(title)

    for i, _y in enumerate(top_y):
        lr = LinearRegression().fit(_y, x)
        score = lr.score(_y, x)

        avg_diffs = []
        avg_diffs_test = []
        for _i, _k in enumerate(_train_keys):
            origin_acc = x[_i][0]
            predict_acc = lr.predict([[top_y[i][_i][0]]])[0][0]
            avg_diffs.append(abs(origin_acc - predict_acc))
        for _i, _k in enumerate(_test_keys):
            origin_acc = test_x[_i][0]
            predict_acc = lr.predict([[test_top_y[i][_i][0]]])[0][0]
            avg_diffs_test.append(abs(origin_acc - predict_acc))
            # print(_i, _k)
            print( _k, origin_acc, predict_acc, abs(origin_acc - predict_acc), test_top_y[i][_i][0])

        diff = sum(avg_diffs)/len(avg_diffs)
        axs[int(i/2)][i%2].scatter(nums, diff*100)
        df[i][0].append(nums)
        df[i][1].append(diff*100)
        test_diff = sum(avg_diffs_test)/len(avg_diffs_test)
        axs[int(i/2)][i%2].scatter(nums, test_diff*100)
        test_df[i][0].append(nums)
        test_df[i][1].append(test_diff*100)
    # plt.show()
    # plt.clf()
    return df, test_df, axs

In [ ]:
def get_train_test_error(cls_result_path, train_list, test_list, title, cls_max, cls_min, step, tops=4, debug=False, save=False):
    df = [[[], []] for _ in range(tops)]
    test_df = [[[], []] for _ in range(tops)]
    axs = None

    for i in range(cls_min, cls_max+1, step):
        df, test_df, axs = classfication_voting(cls_result_path, train_list=train_list, test_list=test_list, tops=4, df=df, test_df=test_df, axs=axs, nums=i, title=title, debug=debug)

    for i in range(tops):
        axs[int(i/2)][i%2].plot(df[i][0], df[i][1], label='Train error')
        axs[int(i/2)][i%2].plot(test_df[i][0], test_df[i][1], label='Test error')
    plt.legend()
    # plt.show()
    plt.clf()
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))
    fig.suptitle(title)

    for i in range(tops):
        axs[0].plot(df[i][0][:], df[i][1][:], label=f'Top {i+1} Train')

    axs[0].legend()
    axs[0].set_ylabel('differences')
    axs[0].set_xlabel('Classfications')

    for i in range(tops):
        axs[1].plot(test_df[i][0][:], test_df[i][1][:], label=f'Top {i+1} Test')
    axs[1].legend()
    axs[1].set_ylabel('differences %')
    axs[1].set_xlabel('Classfications')
    if save:
        plt.savefig(title + '.png', bbox_inches='tight', facecolor='w')
        plt.clf()
    else:
        plt.show()
    
    return df, test_df

In [ ]:
def compare_classification_top_value(cls_result_path, train_list, test_list, tops=4, title='Default', debug=False, save=False):
    df = [[[], []] for _ in range(tops)]
    test_df = [[[], []] for _ in range(tops)]
    df, test_df, axs = classfication_voting(cls_result_path, train_list=train_list, test_list=test_list, tops=tops, df=df, test_df=test_df, title=title, debug=debug)

    plt.clf()
    plt.figure(figsize=(8, 4))
    x = []
    y = []
    test_y = []
    for i, _ in enumerate(df):
        x.append(i+1)
        y.append(df[i][1])
        test_y.append(test_df[i][1])
    
    plt.plot(x, y, label=f'Train error')
    plt.plot(x, test_y, label=f'Test error')
    plt.legend()
    plt.ylabel('differences %')
    plt.xlabel('top')
    plt.title(title)
    if save:
        plt.savefig(title + '.png', bbox_inches='tight', facecolor='w')
        plt.clf()
    else:
        plt.show()
    return y, test_y

In [ ]:

def leave_one_out_test(cls_result_path, dataset_list, tops=4, title='default', cls_tops=4, cls_max=21, cls_min=9, step=2, debug=False, save=False):
    _cls_results = json.load(open(cls_result_path))
    _keys = list(_cls_results['ground_truth'].keys())
    for dataset in dataset_list:
        if dataset not in _keys:
            raise ValueError("Dataset didn't exist!")
    all_y = []
    all_test_y = []

    cls_df = []
    cls_test_df = []
    dataset_list = np.asarray(dataset_list, dtype=str)
    loo = LeaveOneOut()
    print("Total split(s): ", loo.get_n_splits(dataset_list))
    
    for train_index, test_index in loo.split(dataset_list):
        print("TRAIN: ", train_index, "TEST: ", test_index)
        # Top 
        y, test_y = compare_classification_top_value(cls_result_path, dataset_list[train_index], dataset_list[test_index], tops=tops, title='A_' + dataset_list[test_index[0]], debug=debug, save=save)
        all_y.append(y)
        all_test_y.append(test_y)
        # Classfication
        df, test_df = get_train_test_error(cls_result_path, dataset_list[train_index], dataset_list[test_index], title='B_' + dataset_list[test_index[0]], cls_max=cls_max, cls_min=cls_min, step=step, tops=cls_tops, debug=debug, save=save)
        cls_df.append(df)
        cls_test_df.append(test_df)

    # Classfication

    cls_df = np.array(cls_df).sum(axis=0) / len(dataset_list)
    cls_test_df = np.array(cls_test_df).sum(axis=0) / len(dataset_list)

    plt.clf()
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))
    fig.suptitle(title)

    for i in range(cls_tops):
        axs[0].plot(cls_df[i][0][:], cls_df[i][1][:], label=f'Top {i+1} Train')

    axs[0].legend()
    axs[0].set_ylabel('differences')
    axs[0].set_xlabel('Classfications')

    for i in range(cls_tops):
        axs[1].plot(cls_test_df[i][0][:], cls_test_df[i][1][:], label=f'Top {i+1} Test')
    axs[1].legend()
    axs[1].set_ylabel('differences %')
    axs[1].set_xlabel('Classfications')
    if save:
        plt.savefig('Average_cls.png', bbox_inches='tight', facecolor='w')
        plt.clf()
    else:
        plt.show()

    # Top
    avg_y = np.array(all_y).sum(axis=0) / len(dataset_list)
    avg_test_y = np.array(all_test_y).sum(axis=0) / len(dataset_list)
    
    plt.clf()
    plt.figure(figsize=(8, 4))
    x = range(1, tops + 1)
    plt.plot(x, avg_y, label=f'Train error')
    plt.plot(x, avg_test_y, label=f'Test error')
    print("X: ", x)
    print("AVG_Y:", avg_y)
    print("AVG_TEST_Y:", avg_test_y)
    plt.legend()
    plt.ylabel('differences %')
    plt.xlabel('top')
    plt.title(title + '_Average')
    if save:
        plt.savefig('Average.png', bbox_inches='tight', facecolor='w')
        plt.clf()
    else:
        plt.show()

In [ ]:
leave_one_out_test(os.path.join(DATA_DIR, 'final/MCQ/sync_FMA-C-1_2021-02-13_11_SCNN18_SCNN-FMA-C-1-fixed-train_h5_2GPU_cls_result.json'),
    dataset_list=[
        # ---- Test
        'SCNN-Jamendo-test.h5',
        'SCNN-FMA-C-1-fixed-test.h5',
        'SCNN-FMA-C-2-fixed-test.h5',
        'SCNN-KTV-test.h5',
        'SCNN-Taiwanese-CD-test.h5',
        'SCNN-Taiwanese-stream-test.h5',
        'SCNN-Chinese-CD-test.h5',
        'SCNN-Classical-test.h5',
        # ---- Only have one
        'SCNN-MIR-1k-train.h5',
        'SCNN-Instrumental-non-vocal.h5',
        'SCNN-A-Cappella-vocal.h5',
        'SCNN-test-hard.h5',
    ],
    tops = 4,
    title='Default',
    debug=True,
    save=False,
    cls_min=21,
    cls_max=21,
    cls_tops=4,
    step=2,
)

In [ ]:
get_train_test_error('final/MCQ/sync_FMA-C-1_2021-02-13_11_SCNN18_SCNN-FMA-C-1-fixed-train_h5_2GPU_cls_result.json', [
        # ---- Test
        'SCNN-Jamendo-test.h5',
        'SCNN-FMA-C-1-fixed-test.h5',
        'SCNN-FMA-C-2-fixed-test.h5',
        'SCNN-KTV-test.h5',
        'SCNN-Taiwanese-CD-test.h5',
        'SCNN-Taiwanese-stream-test.h5',
        'SCNN-Chinese-CD-test.h5',
        'SCNN-Classical-test.h5',
        # ---- Only have one
        'SCNN-MIR-1k-train.h5',
        'SCNN-Instrumental-non-vocal.h5',
        'SCNN-A-Cappella-vocal.h5',
        'SCNN-test-hard.h5',
    ], ['SCNN-RWC.h5'], cls_max=21, cls_min=21, step=2, tops=4, debug=True, save=False, title='test')

In [ ]:
get_train_test_error('final/MCQ/sync_FMA-C-1_2021-02-13_11_SCNN18_SCNN-FMA-C-1-fixed-train_h5_2GPU_cls_result.json', [
        # ---- Test
        'SCNN-Jamendo-test.h5',
        'SCNN-FMA-C-1-fixed-test.h5',
        'SCNN-FMA-C-2-fixed-test.h5',
        'SCNN-KTV-test.h5',
        # 'SCNN-Taiwanese-CD-test.h5',
        # 'SCNN-Taiwanese-stream-test.h5',
        # 'SCNN-Chinese-CD-test.h5',
        # 'SCNN-Classical-test.h5',
        # ---- Only have one
        'SCNN-MIR-1k-train.h5',
        'SCNN-Instrumental-non-vocal.h5',
        'SCNN-A-Cappella-vocal.h5',
        'SCNN-test-hard.h5',
    ], [
        # # ---- Test
        # 'SCNN-Jamendo-test.h5',
        # 'SCNN-FMA-C-1-fixed-test.h5',
        # 'SCNN-FMA-C-2-fixed-test.h5',
        # 'SCNN-KTV-test.h5',
        'SCNN-Taiwanese-CD-test.h5',
        'SCNN-Taiwanese-stream-test.h5',
        'SCNN-Chinese-CD-test.h5',
        'SCNN-Classical-test.h5',
        # ---- Only have one
        # 'SCNN-MIR-1k-train.h5',
        # 'SCNN-Instrumental-non-vocal.h5',
        # 'SCNN-A-Cappella-vocal.h5',
        # 'SCNN-test-hard.h5',
    ], cls_max=21, cls_min=21, step=2, tops=4, debug=True, save=False, title='test')

In [ ]:
jamendo_acc_rtq = [
0.9402880192,
0.8255813897,
0.8579096556,
0.9403072417,
0.8897533298,
0.8260428429,
0.9105074108,
0.8206219256,
0.8696308136,
0.8019617081,
0.9476521134,
0.6601857483,
]

jamendo_rtq = [
0.8134461343,
0.9082952678,
0.9066032767,
0.6797484457,
0.6485662937,
0.8276259661,
0.8641012132,
0.6629108369,
0.8367223144,
0.6772853196,
0.8127648413,
0.9150333107,
]

jamendo_acc_mcq = [
    0.9375498357,
0.8239435799,
0.8572675415,
0.9401539694,
0.8878648233,
0.8261663177,
0.9100006415,
0.81470657,
0.8702013287,
0.7971706573,
0.9501869417,
0.6614286289,
]


jamendo_mcq = [
0.9082384461,
0.805630355,
0.8424612699,
0.9056327725,
0.8785578748,
0.821307779,
0.9101930849,
0.8191489362,
0.8226837061,
0.8436911488,
0.9324665489,
0.6819442769,
]

jamendo_acc_bzq = [
0.9402880192,
0.8255813897,
0.8579096556,
0.9403072417,
0.8897533298,
0.8260428429,
0.9105074108,
0.8206219256,
0.8696308136,
0.8019617081,
0.9476521134,
0.6601857483,
]

jamendo_bzq = [
0.133088,
0.221322,
0.192189,
0.089027,
0.15351,
0.173506,
0.120251,
0.16117,
0.191427,
0.255446,
0.077291,
0.313183,
]

In [ ]:
np.corrcoef(jamendo_acc_rtq, jamendo_rtq)

In [ ]:
fmac1_acc_rtq = [
0.928466171,
0.8968665957,
0.9080078483,
0.9568397999,
0.9383301735,
0.8878241241,
0.9299506009,
0.8950491011,
0.8933262348,
0.8710138142,
0.942426151,
0.7259317398,
]

fmac1_rtq = [
0.7009548008,
0.86113922,
0.8570181191,
0.5442712516,
0.525697735,
0.6910967827,
0.7558185041,
0.6676432133,
0.7884119511,
0.6505630851,
0.7013464987,
0.8453566194,
]

fmac1_acc_mcq = [
0.9273434759,
0.8962872297,
0.906954272,
0.956143101,
0.9345802837,
0.8868577871,
0.9284110591,
0.8934416647,
0.8916611728,
0.8719023705,
0.9433103714,
0.7243884648,
]


fmac1_mcq = [
0.8901540522,
0.8641370869,
0.8876281911,
0.9312362838,
0.8918406072,
0.8675310034,
0.9164795689,
0.8711129296,
0.8548100816,
0.8769617075,
0.9248927039,
0.7136654203,
]

fmac1_acc_bzq = [
0.928466171,
0.8968665957,
0.9080078483,
0.9568397999,
0.9383301735,
0.8878241241,
0.9299506009,
0.8950491011,
0.8933262348,
0.8710138142,
0.942426151,
0.7259317398,
]

fmac1_bzq = [
0.121735,
0.144088,
0.124395,
0.069788,
0.10019,
0.135457,
0.089044,
0.123241,
0.140327,
0.153311,
0.0859,
0.255084,
]

In [ ]:
np.corrcoef(fmac1_acc_mcq, fmac1_mcq)

In [ ]:
lr = LinearRegression()


lr.fit(np.array(jamendo_mcq).reshape(-1, 1), np.array(jamendo_acc_mcq).reshape(-1, 1))

In [ ]:
lr = LinearRegression()


lr.fit(np.array(jamendo_mcq).reshape(-1, 1), np.array(jamendo_acc_mcq).reshape(-1, 1))
score = lr.score(np.array(jamendo_mcq).reshape(-1, 1), np.array(jamendo_acc_mcq).reshape(-1, 1))
plt.title(f'MCQ Top 4 \nscore: {score:.4f}')
plt.scatter(jamendo_acc_mcq, jamendo_mcq)
line = np.arange(0, 1, 0.001).reshape(-1, 1)
plt.plot(lr.predict(line), line, linewidth=1)
plt.xlabel('Accuracy')
plt.ylabel('MCQ Top 4')
plt.axis([min(jamendo_acc_mcq)-0.05, max(jamendo_acc_mcq)+0.05, min(jamendo_mcq)-0.05, max(jamendo_mcq)+0.05])

In [ ]:
lr = LinearRegression()


lr.fit(np.array(fmac1_rtq).reshape(-1, 1), np.array(fmac1_acc_rtq).reshape(-1, 1))
score = lr.score(np.array(fmac1_rtq).reshape(-1, 1), np.array(fmac1_acc_rtq).reshape(-1, 1))
plt.title(f'FMA-C-1 RTQ \nscore: {score:.4f}')
plt.scatter(fmac1_acc_rtq, fmac1_rtq)
line = np.arange(0, 1, 0.001).reshape(-1, 1)
plt.plot(lr.predict(line), line, linewidth=1)
plt.xlabel('Accuracy')
plt.ylabel('FMA-C-1 RTQ')
plt.axis([min(fmac1_acc_rtq)-0.05, max(fmac1_acc_rtq)+0.05, min(fmac1_rtq)-0.05, max(fmac1_rtq)+0.05])